<a href="https://colab.research.google.com/github/Cado87/Fruit-classification/blob/main/Fruit_classifier_dataset_ethanjxu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Download dataset

New dataset for fine-tuning(https://huggingface.co/datasets/ethanjxu/fruit_images)

In [1]:
from datasets import load_dataset

ds = load_dataset("ethanjxu/fruit_images")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/423M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/425M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/152M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/90580 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/15985 [00:00<?, ? examples/s]